<a href="https://colab.research.google.com/github/rts1988/Duolingo_spaced_repetition/blob/main/BNC_corpus_sum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BNC corpus combining from individual frequency dictionaries 
##Combine all the csv files into one, and calculate the document frequency of each word. 
## Also compute the frequency of the non-lemmatized forms of each word
## Compare with Reuters idf dict

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [ ]:
path = "/content/drive/MyDrive/BNC_lemmafiles/"
os.chdir(path)
fileids = os.listdir()

def forms_freq(listforms):
  forms_dict=dict()
  for lf in listforms:
    if lf in forms_dict:
      forms_dict[lf] +=1
    else:
      forms_dict[lf] = 1
  return [(k, v) for k, v in forms_dict.items()]

In [ ]:
len(fileids)

4050

In [ ]:
import numpy as num
fileidslist = [fileids[i:i+20] for i in num.arange(start=0,stop=len(fileids),step=20)]


In [ ]:
len(fileidslist)

203

In [ ]:
import pandas as pd
df = pd.read_csv('H_HU_HU1_count9459463_lemmacount.csv')

In [ ]:
df['forms'] = df['forms'].apply(eval)
df.columns = ['lemma','count','forms']
df['forms_freq'] = df['forms'].apply(forms_freq)
alldf = df.copy()
alldf['doc_count'] = 1
del alldf['forms']

In [ ]:
alldf.head()
#del alldf['forms']

,lemma,count,forms_freq,doc_count
0,the,1343,"[(The, 204), (the, 1128), (THE, 11)]",1
1,1993,21,"[(1993, 21)]",1
2,conference,28,"[(Conference, 14), (conferences, 1), (conferen...",1
3,—,32,"[(—, 32)]",1
4,peebles,8,"[(Peebles, 8)]",1


In [ ]:
count = 0
for fileid in fileids[1:]:
  print(fileid)
  df = pd.read_csv(fileid)
  df['forms'] = df['forms'].apply(eval)
  df.columns = ['lemma','count','forms']
  df['forms_freq'] = df['forms'].apply(forms_freq)
  for i,row in df.iterrows():
    subdf = alldf.loc[alldf['lemma']==row['lemma'],:]
    if len(subdf)>0:
      alldf.loc[alldf['lemma']==row['lemma'],'doc_count'] +=1
      alldf.loc[alldf['lemma']==row['lemma'],'count'] = subdf['count'] + row['count']
      
      #allforms_freq = alldf.loc[alldf['lemma']==row['lemma'],'forms_freq']
      allforms_dict = dict(zip([tup[0] for tup in row['forms_freq']],[tup[1] for tup in row['forms_freq']]))
      newforms_freq = row['forms_freq']
      newforms_dict = dict(zip([tup[0] for tup in newforms_freq],[tup[1] for tup in newforms_freq]))
      allforms_keys = set([k for (k,v) in allforms_dict.items()])
      newforms_keys = set([k for (k,v) in newforms_dict.items()])
      common_keys = allforms_keys.intersection(newforms_keys)
      new_keys = newforms_keys.difference(allforms_keys)
      for ck in common_keys:
        allforms_dict[ck] = allforms_dict[ck] + newforms_dict[ck]
      for nk in new_keys:
        allforms_dict[nk] = newforms_dict[nk]
      alldf.loc[alldf['lemma']==row['lemma'],'forms_freq'].apply(lambda _:[(k,v) for (k,v) in allforms_dict.items()].copy())
    else:
      alldf = alldf.append({'lemma':row['lemma'],'count':row['count'],'forms_freq':row['forms_freq'],'doc_count':1},ignore_index=True)
  count +=1
  if count%10==0:
    print('finished ',count)
    

      


H_HU_HU2_count9469247_lemmacount.csv
H_HU_HU3_count9479258_lemmacount.csv
H_HU_HU4_count9489332_lemmacount.csv
H_HU_HU5_count9491476_lemmacount.csv
H_HU_HU6_count9492791_lemmacount.csv
H_HU_HU7_count9494459_lemmacount.csv
H_HU_HU8_count9496569_lemmacount.csv
H_HU_HU9_count9499479_lemmacount.csv
H_HU_HUA_count9501659_lemmacount.csv
H_HU_HUB_count9504786_lemmacount.csv
finished  10
H_HU_HUC_count9506422_lemmacount.csv
H_HU_HUD_count9507888_lemmacount.csv
H_HU_HUE_count9509808_lemmacount.csv
H_HU_HUF_count9510771_lemmacount.csv
H_HU_HUG_count9511208_lemmacount.csv
H_HU_HUH_count9512430_lemmacount.csv
H_HU_HUJ_count9514143_lemmacount.csv
H_HU_HUK_count9515791_lemmacount.csv
H_HU_HUL_count9517301_lemmacount.csv
H_HU_HUM_count9519029_lemmacount.csv
finished  20
H_HU_HUN_count9520746_lemmacount.csv
H_HU_HUP_count9522140_lemmacount.csv
H_HU_HUR_count9522495_lemmacount.csv
H_HU_HUS_count9522850_lemmacount.csv
H_HU_HUT_count9523770_lemmacount.csv
H_HU_HUU_count9524696_lemmacount.csv
H_HU_HUV_cou

KeyboardInterrupt: ignored

In [ ]:
fileids.index(fileid)

708

In [ ]:
#dict(zip([tup[0] for tup in row['forms_freq']],[tup[1] for tup in row['forms_freq']]))
alldf.to_csv('BNC_DICT.csv')
from google.colab import files
files.download('BNC_DICT.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

dnaq's answer on multiprocessing and queueing 
https://stackoverflow.com/questions/12538485/python-multiprocess-update-dictionary-synchronously

In [ ]:
# from multiprocessing.pool import ThreadPool as Pool
# # from multiprocessing import Pool

# pool_size = 5  # your "parallelness"

# # define worker function before a Pool is instantiated
# def worker(item):
#     try:
#         api.my_operation(item)
#     except:
#         print('error with item')

# pool = Pool(pool_size)

# for item in items:
#     pool.apply_async(worker, (item,))

# pool.close()
# pool.join()
import datetime



import multiprocessing.pool
concurrent = 20

def run_repeatedly(item):
  # doing something here
  return item+'done'+str(datetime.datetime.now())


def main():
  stringlist = ['arati']*100 + ['vaibhav']*100 + ['shankar']*100

  # a thread pool that implements the process pool API.
  pool = multiprocessing.pool.ThreadPool(processes=concurrent)
  return_list = pool.map(run_repeatedly, stringlist, chunksize=100)
  pool.close()
  for data in return_list:
    print(data)
  

main()

In [ ]:
# each process will run this function
import pandas as pd

def forms_freq(listforms):
  forms_dict=dict()
  for lf in listforms:
    if lf in forms_dict:
      forms_dict[lf] +=1
    else:
      forms_dict[lf] = 1
  return [(k, v) for k, v in forms_dict.items()]

def combine(fileids):
  #print(fileids)
  
  def combine_lists(serieslist):
    alllist = []
    for sl in serieslist:
      alllist = alllist + sl
    return alllist

  count = 0
  alldf = pd.DataFrame()
  combineddf = pd.DataFrame()
  for f in fileids:
    try:
      df = pd.read_csv(f)
    except:
      print('could not read ',f)
    try:
      df['forms'] = df['forms'].apply(eval)
    except:
      print('errored out on forms eval',f)
    try:
      df.columns = ['lemma','count','forms']
    except:
      print('errored out on column renaming',f)
    try:
      alldf = alldf.append(df,ignore_index=True)
    except:
      print('errored out on appending df to alldf',f)
    count +=1
    print('done file: ',f)
  #print(len(alldf))
  print('loaded all files, groupby operations starting')
  try:
    combineddf['lemma_count'] = alldf.groupby('lemma')['count'].sum()
    combineddf['doc_count'] = alldf['lemma'].value_counts()
    combineddf['allforms'] = alldf.groupby('lemma')['forms'].apply(combine_lists)
    combineddf['formsfreq'] = combineddf['allforms'].apply(forms_freq)
  except:
    print('Erroring out on combining the dfs',fileids)
  del combineddf['allforms'],alldf,df
  combineddf = combineddf.reset_index()
  print('Done chunk ',fileids[-1])
  return combineddf

In [ ]:
combineddf1 = combine(fileidslist[0])

done file:  H_HU_HU2_count9469247_lemmacount.csv
done file:  H_HU_HU3_count9479258_lemmacount.csv
done file:  H_HU_HU4_count9489332_lemmacount.csv
done file:  H_HU_HU5_count9491476_lemmacount.csv
done file:  H_HU_HU6_count9492791_lemmacount.csv
done file:  H_HU_HU7_count9494459_lemmacount.csv
done file:  H_HU_HU8_count9496569_lemmacount.csv
done file:  H_HU_HU9_count9499479_lemmacount.csv
done file:  H_HU_HUA_count9501659_lemmacount.csv
done file:  H_HU_HUB_count9504786_lemmacount.csv
done file:  H_HU_HUC_count9506422_lemmacount.csv
done file:  H_HU_HUD_count9507888_lemmacount.csv
done file:  H_HU_HUE_count9509808_lemmacount.csv
done file:  H_HU_HUF_count9510771_lemmacount.csv
done file:  H_HU_HUG_count9511208_lemmacount.csv
done file:  H_HU_HUH_count9512430_lemmacount.csv
done file:  H_HU_HUJ_count9514143_lemmacount.csv
done file:  H_HU_HUK_count9515791_lemmacount.csv
done file:  H_HU_HUL_count9517301_lemmacount.csv
done file:  H_HU_HUM_count9519029_lemmacount.csv
loaded all files, gr

In [ ]:
combineddf1

,lemma,lemma_count,doc_count,formsfreq
0,!,169,9,"[(!, 169)]"
1,"$1,340",1,1,"[($1,340, 1)]"
2,$1099.00,1,1,"[($1099.00, 1)]"
3,$1148.00,1,1,"[($1148.00, 1)]"
4,$327,1,1,"[($327, 1)]"
...,...,...,...,...
25565,≤4,2,1,"[(≤4, 2)]"
25566,≤50000/mm,1,1,"[(≤50000/mm, 1)]"
25567,≤6,1,1,"[(≤6, 1)]"
25568,≤90%,1,1,"[(≤90%, 1)]"


In [ ]:
import multiprocessing.pool
concurrent = 203

# def run_repeatedly(item):
#   # doing something here
#   return item+'done'+str(datetime.datetime.now())


def main():

  # a thread pool that implements the process pool API.
  pool = multiprocessing.pool.ThreadPool(processes=concurrent)
  return_list = pool.map(combine, fileidslist, chunksize=1)
  pool.close()

  combineddata = pd.DataFrame()
  for data in return_list:
    combineddata = combineddata.append(data,ignore_index=True)
  return combineddata

combineddata = main()
combineddata.to_csv('BNC_dict20211024_2.csv')

In [ ]:
def combine_lists(serieslist):
  alllist = []
  for sl in serieslist:
    alllist = alllist + sl
  return alllist

In [ ]:
comb2round = pd.DataFrame()
comb2round['lemma_count'] = combineddata.groupby('lemma')['lemma_count'].sum()
comb2round['doc_count'] = combineddata.groupby('lemma')['doc_count'].sum()
comb2round['formsfreq'] = combineddata.groupby('lemma')['formsfreq'].apply(combine_lists)
comb2round.head(3)

,lemma_count,doc_count,formsfreq
lemma,,,
!,141559,2493,"[(!, 169), (!, 3), (!, 414), (!, 784), (!, 158..."
! $%;,1,1,"[(! $%;, 1)]"
"""",141554,368,"[("", 7537), ("", 7148), ("", 1), ("", 1834), ("", ..."


In [ ]:
def combformfreq(listseries):
  listtup = combine_lists(listseries) 
  freqdict = dict()
  for tup in listtup:
    if tup[0] in freqdict:
      freqdict[tup[0]] = freqdict[tup[0]] + tup[1]
    else:
      freqdict[tup[0]] = tup[1]
  tuple_list = [(k,v) for (k,v) in freqdict.items()]
  return tuple_list



In [ ]:
comb2round['formsfreq'] = combineddata.groupby('lemma')['formsfreq'].apply(combformfreq)

In [ ]:
comb2round.head(10)

,lemma_count,doc_count,formsfreq
lemma,,,
!,141559,2493,"[(!, 141559)]"
! $%;,1,1,"[(! $%;, 1)]"
"""",141554,368,"[("", 141554)]"
#,5,3,"[(#, 5)]"
#1,2,2,"[(#1, 2)]"
"#150,000,000",1,1,"[(#150,000,000, 1)]"
#5,1,1,"[(#5, 1)]"
#6,1,1,"[(#6, 1)]"
#elected,1,1,"[(#Elected, 1)]"


In [ ]:
import math
comb2round['IDF'] = comb2round['doc_count'].apply(lambda x: math.log(4045/(1 + x),2))

In [ ]:
comb2round.head(20)

,lemma_count,doc_count,formsfreq,IDF
lemma,,,,
!,141559,2493,"[(!, 141559)]",0.697678
! $%;,1,1,"[(! $%;, 1)]",10.981924
"""",141554,368,"[("", 141554)]",3.454447
#,5,3,"[(#, 5)]",9.981924
#1,2,2,"[(#1, 2)]",10.396961
"#150,000,000",1,1,"[(#150,000,000, 1)]",10.981924
#5,1,1,"[(#5, 1)]",10.981924
#6,1,1,"[(#6, 1)]",10.981924
#elected,1,1,"[(#Elected, 1)]",10.981924


In [ ]:
filename = 'BNC_IDFdict.csv'
comb2round.to_csv(filename)


lemma_count                                    539
doc_count                                       82
formsfreq      [(Beth, 535), (BETH, 2), (beth, 2)]
IDF                                        5.60688
Name: beth, dtype: object

In [ ]:
math.log(4045,2)

11.98192398731734

In [ ]:
%%time
combineddf = combine(fileids)

100 docs done
200 docs done


KeyboardInterrupt: ignored